In [1]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score, root_mean_squared_error

## Carregar e visualizar os dados  - Regressão


In [2]:
df_colesterol = pd.read_csv('./dataset_colesterol.csv')

In [3]:
df_colesterol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [4]:
df_colesterol.describe()

,Id,Idade,Peso,Altura,Colesterol
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,49.160000,76.487500,174.868000,199.453260
std,288.819436,17.494755,21.411086,8.855337,50.625152
min,1.000000,20.000000,15.400000,160.000000,36.260000
25%,250.750000,34.000000,61.775000,167.000000,166.345000
50%,500.500000,48.500000,75.450000,175.000000,199.455000
75%,750.250000,64.250000,89.975000,183.000000,233.785000
max,1000.000000,79.000000,158.100000,190.000000,372.670000


In [5]:
df_colesterol.head()

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33,85.1,186,199.63
1,2,A,Não,Moderado,68,105.0,184,236.98
2,3,O,Não,Alto,25,64.8,180,161.79
3,4,A,Não,Alto,43,120.2,167,336.24
4,5,AB,Não,Baixo,79,88.5,175,226.23


In [6]:
import plotly.express as px

px.bar(df_colesterol.value_counts('Nível de Atividade')/len(df_colesterol) * 100)

In [7]:
px.bar(df_colesterol.value_counts('Fumante')/len(df_colesterol) * 100)

In [8]:
px.bar(df_colesterol.value_counts('Grupo Sanguíneo')/len(df_colesterol) * 100)

In [9]:
# Ajustar o Dataframe
df_colesterol.drop(columns=['Id'],axis=1,inplace=True)

# Aplicar OneHotEncoding nas variáveis Categoricas
df_colesterol = pd.get_dummies(df_colesterol,columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade']).astype("int64")

In [10]:
df_colesterol.head()

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85,186,199,0,0,1,0,0,1,0,1,0
1,68,105,184,236,1,0,0,0,1,0,0,0,1
2,25,64,180,161,0,0,0,1,1,0,1,0,0
3,43,120,167,336,1,0,0,0,1,0,1,0,0
4,79,88,175,226,0,1,0,0,1,0,0,1,0


## Treinar modelo de Regressão Linear multipla com RFE

In [11]:
X = df_colesterol.drop('Colesterol', axis=1)
y = df_colesterol['Colesterol']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [13]:
# Treinar o modelo com RFE
# RFE (Recursive Feature Elimination)
# Uso um estimador e defino uma quantidade de feature (dois hiperparâmetros)
# O RFE faz iterações iniciando com todas as features e eliminando a cada iteração até atingir a qtde. definida
# Elimina as caracteristicas/Features menos importantes

rfe_method = RFE(estimator=LinearRegression(),n_features_to_select=6)
rfe_method.fit(X_train,y_train)

,estimator,LinearRegression()
,n_features_to_select,6
,step,1
,verbose,0
,importance_getter,'auto'
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [14]:
# Quais features foram selecionadas?
# rfe_method.get_support() nos dá uma lista dos índices das columas que foram escolhidas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto'],
      dtype='object')

In [15]:
# Ranking de Features
def mostrar_ranking(metodo_fs, X_train):

    # Obter o Ranking de features
    ranking = metodo_fs.ranking_

    # Obter os nomes das features
    nomes_features = X_train.columns.to_list()

    # Crie um Dataframe com os rankings e os nomes das features
    df_ranking = pd.DataFrame({'Feature': nomes_features, 'Ranking': ranking})

    # Ordene o Dataframe pelo Ranking
    df_ranking = df_ranking.sort_values(by='Ranking')

    # Exibir o ranking
    print(df_ranking)

In [16]:
# Ranking de Features do RFE Regressão
mostrar_ranking(rfe_method,X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
7                   Fumante_Não        1
4            Grupo Sanguíneo_AB        1
9       Nível de Atividade_Alto        1
8                   Fumante_Sim        1
5             Grupo Sanguíneo_B        2
3             Grupo Sanguíneo_A        3
11  Nível de Atividade_Moderado        4
10     Nível de Atividade_Baixo        5
6             Grupo Sanguíneo_O        6
0                         Idade        7


In [17]:
# Função para Avaliar performance

def performance_regressao(modelo, X_test, y_test):

    # Faz a predição com o modelo no conjunto de testes
    y_pred = modelo.predict(X_test)

    # Avalaiar desempenho
    return root_mean_squared_error(y_test,y_pred)

In [18]:
# Performance Regressão com RFE
performance_regressao(rfe_method,X_test, y_test)

8.988758758123277

## Treinar modelo sem RFE

In [19]:
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [20]:
performance_regressao(model_reg,X_test,y_test)

9.060086735126106

Observe que dependendo do numero de features escolhidas, no meu caso 6, você consegue uma performance melhor no modelo RFE

## Treinar modelo de Regressão Linear com RFECV

In [21]:
# utiliza validação cruzada (cv)
# no mínimo 6, podem ter mais do que 6
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train,y_train)

,estimator,LinearRegression()
,step,1
,min_features_to_select,6
,cv,5
,scoring,None
,verbose,0
,n_jobs,None
,importance_getter,'auto'
,fit_intercept,True
,copy_X,True
,tol,1e-06


In [22]:
performance_regressao(rfe_method_cv,X_test,y_test)

9.065068454304681

In [23]:
# Quais features foram escolhidas?
X_train.columns[(rfe_method_cv.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [24]:
# Quantas features foram selecionadas?
rfe_method_cv.n_features_

np.int64(10)

## Treinar modelo de regressão com SelectFromModel

In [25]:
# SelectFromModel n tem um método predict, estaão vamos passar o modelo que treinamos mais acima

sfm_method = SelectFromModel(estimator=model_reg, max_features=4,threshold=0.5)
sfm_method.fit(X_train,y_train)

,estimator,LinearRegression()
,threshold,0.5
,prefit,False
,norm_order,1
,max_features,4
,importance_getter,'auto'
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [26]:
X_train.columns[(sfm_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Nível de Atividade_Alto'], dtype='object')

In [27]:
# Treinar o modelo com as features selecionadas

X_train_ajustado_reg = sfm_method.transform(X_train)
X_test_ajustado_reg = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado_reg, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


O método transform de uma instância de SelectFromModel (sfm no seu código) executa uma operação simples e crucial:

Ele pega o dataset original de features (por exemplo, X_train).

Ele filtra esse dataset para incluir somente as colunas que o SelectFromModel (sfm) selecionou após ser treinado (sfm.fit(X_train, y_train)).

No seu caso, a linha de código X_train.columns[[sfm_method.get_support()]] mostrou que as features selecionadas foram 'Peso' e 'Altura'.

In [28]:
# Performance do modelo com selectfrommodel
performance_regressao(model_reg,X_test_ajustado_reg,y_test)

9.062806089261787

## Calcular PPS - Classificação

In [29]:
df2 = pd.read_csv('./fruit_quality.csv')

In [30]:
df2.drop('A_id', axis=1, inplace=True)
df2['Quality'] = (df2["Quality"] == "good").astype('int32')

df2.head()

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1


## Treinar modelo de Regressão Logistica com RFE

In [31]:
X = df2.drop('Quality', axis=1)
y = df2['Quality']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [33]:
# Treinar o modelo com RFE
# RFE (Recursive Feature Elimination)
# Uso um estimador e defino uma quantidade de feature (dois hiperparâmetros)
# O RFE faz iterações iniciando com todas as features e eliminando a cada iteração até atingir a qtde. definida
# Elimina as caracteristicas/Features menos importantes

rfe_method = RFE(estimator=LogisticRegression(),n_features_to_select=5)
rfe_method.fit(X_train,y_train)

,estimator,LogisticRegression()
,n_features_to_select,5
,step,1
,verbose,0
,importance_getter,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1


In [34]:
# Quais features foram selecionadas?
# rfe_method.get_support() nos dá uma lista dos índices das columas que foram escolhidas
X_train.columns[(rfe_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [35]:
# Ranking de Features do RFE Regressão
mostrar_ranking(rfe_method,X_train)

       Feature  Ranking
0         Size        1
1       Weight        1
2    Sweetness        1
4    Juiciness        1
6      Acidity        1
5     Ripeness        2
3  Crunchiness        3


In [36]:
# Função para Avaliar performance

def performance_classificacao(modelo, X_test, y_test):

    # Faz a predição com o modelo no conjunto de testes
    y_pred = modelo.predict(X_test)

    # Avalaiar desempenho
    return root_mean_squared_error(y_test,y_pred)

In [37]:
# Performance Regressão com RFE
performance_classificacao(rfe_method,X_test, y_test)

0.47696960070847283

## Treinar modelo sem RFE

In [38]:
model_lr = LogisticRegression()
model_lr.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [39]:
performance_classificacao(model_lr,X_test,y_test)

0.4716990566028302

## Treinar modelo de Regressão Linear com RFECV

In [40]:
# utiliza validação cruzada (cv)
# no mínimo 2, podem ter mais do que 2
rfe_method_cv = RFECV(estimator=LogisticRegression(), min_features_to_select=2, cv=5, scoring='f1_weighted')
rfe_method_cv.fit(X_train,y_train)

,estimator,LogisticRegression()
,step,1
,min_features_to_select,2
,cv,5
,scoring,'f1_weighted'
,verbose,0
,n_jobs,None
,importance_getter,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001


In [41]:
performance_regressao(rfe_method_cv,X_test,y_test)

0.473462423711393

In [42]:
# Quais features foram escolhidas?
X_train.columns[(rfe_method_cv.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Ripeness', 'Acidity'], dtype='object')

In [43]:
# Quantas features foram selecionadas?
rfe_method_cv.n_features_

np.int64(6)

## Treinar modelo de regressão logistica com SelectFromModel

In [56]:
# SelectFromModel n tem um método predict, estaão vamos passar o modelo que treinamos mais acima

sfm_method = SelectFromModel(estimator=model_lr, max_features=5, threshold=0.01)
sfm_method.fit(X_train,y_train)

,estimator,LogisticRegression()
,threshold,0.01
,prefit,False
,norm_order,1
,max_features,5
,importance_getter,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True


In [57]:
X_train.columns[(sfm_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Acidity'], dtype='object')

In [58]:
# Treinar o modelo com as features selecionadas

X_train_ajustado_lr = sfm_method.transform(X_train)
X_test_ajustado_lr = sfm_method.transform(X_test)
model_lr.fit(X_train_ajustado_lr, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


O método transform de uma instância de SelectFromModel (sfm no seu código) executa uma operação simples e crucial:

Ele pega o dataset original de features (por exemplo, X_train).

Ele filtra esse dataset para incluir somente as colunas que o SelectFromModel (sfm) selecionou após ser treinado (sfm.fit(X_train, y_train)).

No seu caso, a linha de código X_train.columns[[sfm_method.get_support()]] mostrou que as features selecionadas foram 'Peso' e 'Altura'.

In [59]:
# Performance do modelo com selectfrommodel
performance_classificacao(model_lr,X_test_ajustado_lr,y_test)

0.47696960070847283